In [2]:
from utils import *

train_data = read_train_data()
test_data = read_test_data()

In [2]:
from sklearn.feature_extraction.text import TfidfVectorizer

tfidf = TfidfVectorizer()
tfidf.fit(train_data['content'].apply(lambda x: ' '.join(x)))
train_tfidf_feat = tfidf.transform(train_data['content'].apply(lambda x: ' '.join(x)))
test_tfidf_feat = tfidf.transform(train_data['content'].apply(lambda x: ' '.join(x)))

tfidf = TfidfVectorizer(ngram_range=(1,2), max_features=5000)
tfidf.fit(train_data['content'].apply(lambda x: ' '.join(x)))
train_tfidf_feat = tfidf.transform(train_data['content'].apply(lambda x: ' '.join(x)))
test_tfidf_feat = tfidf.transform(train_data['content'].apply(lambda x: ' '.join(x)))

In [4]:
train_data['content'].shape, train_tfidf_feat.shape

((14000,), (14000, 5000))

In [6]:
len(tfidf.get_feature_names_out())

5000

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer

from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report

tfidf = TfidfVectorizer()
tfidf.fit(train_data['content'].apply(lambda x: ' '.join(x)))
train_tfidf_feat = tfidf.transform(train_data['content'].apply(lambda x: ' '.join(x)))
test_tfidf_feat = tfidf.transform(test_data['content'].apply(lambda x: ' '.join(x)))

# 使用cross_val_predict()函数进行交叉验证
val_pred = cross_val_predict(
    LogisticRegression(),
    train_tfidf_feat,
    train_data['label']
)
# 计算并输出分类报告
print(classification_report(train_data['label'], val_pred, digits=3))

              precision    recall  f1-score   support

           0      0.981     0.999     0.990     11836
           1      0.992     0.892     0.940      2164

    accuracy                          0.982     14000
   macro avg      0.986     0.946     0.965     14000
weighted avg      0.982     0.982     0.982     14000



In [5]:
# 生成测试集结果
m = LogisticRegression()
m.fit(train_tfidf_feat, train_data['label'])
test_pred = m.predict(test_tfidf_feat)
test_data['label'] = test_pred
test_data[['name', 'label']].to_csv('lr.csv', index=None)